In [1]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [2]:
from kaggle_secrets import UserSecretsClient

In [4]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [5]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git
    

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 59 (delta 16), reused 41 (delta 6), pack-reused 0
Unpacking objects: 100% (59/59), 27.02 MiB | 6.54 MiB/s, done.


In [6]:
%cd /kaggle/working/llama-3-finetune

/kaggle/working/llama-3-finetune


In [7]:
!git checkout -b Update-Eval-Notebook

Switched to a new branch 'Update-Eval-Notebook'


In [8]:
!git pull origin Update-Eval-Notebook

From https://github.com/shreeya-dhakal/llama-3-finetune
 * branch            Update-Eval-Notebook -> FETCH_HEAD
Updating 0b9d6ce..7258630
Fast-forward
 data_splits/alpaca-cleaned-hindi_test_split.txt  | 10352 +++++
 data_splits/alpaca-cleaned-hindi_train_split.txt | 41408 ++++++++++++++++++++
 data_splits/alpaca-nepali-sft_test_split.txt     | 10392 ++++++
 data_splits/alpaca-nepali-sft_train_split.txt    | 41567 +++++++++++++++++++++
 data_splits/aya_dataset_hindi_test_split.txt     |   231 +
 data_splits/aya_dataset_hindi_train_split.txt    |   922 +
 data_splits/aya_dataset_nepali_test_split.txt    |   801 +
 data_splits/aya_dataset_nepali_train_split.txt   |  3201 ++
 src/__init__.py                                  |     0
 src/data/__init__.py                             |     0
 src/data/data_sampler.py                         |    75 +
 src/notebooks/llama3_eval.ipynb                  |    24 -
 src/notebooks/llama3_finetune.ipynb              |    22 +
 src/notebooks/llama3_in

In [4]:
from datasets import load_dataset
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-07 07:23:14.168561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 07:23:14.168619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 07:23:14.170157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
split_ratio = 0.2
seed = 42

In [7]:
dataset_name = "CohereForAI/aya_dataset"
dataset_type = "aya"

In [8]:
lang_multi = "Nepali"
_, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [10]:
aya_nepali_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 801
})

In [17]:
lang_multi = "Hindi"
_, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

In [18]:
aya_hindi_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 231
})

In [28]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

In [10]:
alpaca_nepali_test

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 10392
})

In [29]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [30]:
alpaca_hindi_test

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 10352
})

In [35]:
alpaca_prompt_hindi = """Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [11]:
alpaca_prompt_nepali = """Below is an instruction in nepali that describes a task, paired with an input also in nepali that provides further context. Write a response in nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [12]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.895 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_inference(dataset, prompt, is_alpaca_format=True):
    for i, row in enumerate(dataset):
        if is_alpaca_format:
            instruction = row["instruction"]
            input_text = row["input"]
        else:
            instruction = row["inputs"]
            input_text = ""

        prompt = prompt.format(instruction, input_text, "")
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs)[0]
        dataset[i]["output"] = generated_text
        print(generated_text)
    return dataset

In [1]:
# alpaca_hindi_test_s = alpaca_hindi_test.select(range(10)) 
# run_inference(alpaca_hindi_test_s, alpaca_prompt_hindi, is_alpaca_format=True)
aya_nepali_test_s = aya_nepali_test.select(range(10)) 
aya_nepali_test_s = run_inference(aya_nepali_test_s, alpaca_prompt_nepali, is_alpaca_format=False)

NameError: name 'aya_nepali_test' is not defined

In [1]:
aya_nepali_test_s.to_csv("aya_nepali_test_s.csv")

NameError: name 'aya_nepali_test_s' is not defined